# Prototyping the Swept Langmuir Diagnostic

This notebook will focus on the mechanics of operating on 2D DataArrays where one dimension is `shot_index` and the second is `time_index`.  **This notebook is for prototyping purposes and will either be deleted or converted into a documentaiton notebook when the protoyping is finished.**

### Table of Contents

1. [Import Packages](#Import-Packages)
1. [Enable the Swept Langmuir Diagnostic](#Enable-the-Swept-Langmuir-Diagnostic)
1. [Build the `xarray.Dataset`](#Build-the-xarray.Dataset)

## Import Packages

In [ ]:
%matplotlib inline

import astropy.units as u
import numpy as np
import os
import pandas as pd
import sys
import xarray as xr

from numbers import Number
from typing import Mapping, Hashable, Any
from warnings import warn
from xarray.core.utils import either_dict_or_kwargs
from xarray.core.coordinates import remap_label_indexers

from plasmapy import __file__ as plasmapy_file
from plasmapy.diagnostics import xdiagnostics
from plasmapy.utils.exceptions import PlasmaPyError, PlasmaPyWarning

plasmapy_dir = os.path.dirname(os.path.abspath(plasmapy_file))

## Enable the Swept Langmuir Diagnostic

By default `plasmapy` does not register any of its `xarray.Dataset` accessors to avoid polution of the namespace and reduce overhead of importing unnecessary code.  Instead, we provide the `xdiagnostics` object so the user can enable the `xarray.Dataset` accessor(s) he or she wants.  Just executing `xdiagnostics` will show what diagnosics are available and enabled/disabled.

In [ ]:
xdiagnostics

In [ ]:
hasattr(xr.Dataset, "swept_langmuir")

In [ ]:
# xdiagnostics.enable('swept_langmuir')
xdiagnostics.enable('swept_langmuir', rename="slang")
xdiagnostics

In [ ]:
(hasattr(xr.Dataset, "swept_langmuir"),
 hasattr(xr.Dataset, "slang"),
)

## Build the `xarray.Dataset`

- The `Dataset` needs to have two `DataArray`s named `voltage` and `current`.
- The `DataArray`'s are either 1D or 2D.
- The `DataArray`'s have to have matching `dims` and `shape`.
- The names of the dimesions do not matter, since the **diagnostic** is agnostic towards the names but the order does matter.
  - Dims for 1D: `dims=["time_index"]`
  - Dims for 2D: `dims=["signal_index", "time_index"]`

In [ ]:
# Load the bias and current values stored in the example file
path = os.path.abspath(os.path.join(
    plasmapy_dir, '..','docs', "notebooks", "langmuir_samples", "Beckers2017.npy"
))
bias, current = np.load(path)
bias = np.tile(bias, (4, 1))
current = np.tile(current, (4, 1))

# voltage DataArray
da_volt = xr.DataArray(bias,
                       dims=['shotnum', 'time'],
                       name='voltage')

# current DataArray
da_curr = xr.DataArray(current,
                       dims=['shotnum', 'time'],
                       name='current')

# Build coordinate arrays
shotnum = np.arange(10, 14)
time = np.arange(bias.shape[1]) * 0.1


# build dataset
ds = xr.Dataset(
    {da_volt.name: da_volt,
     da_curr.name: da_curr},
#     coords={"shotnum": ("shotnum", shotnum),
#             "time": ("time", time)},
        coords={"shotnum": ("shotnum", shotnum)},
)

# initialize Dataset with swept langmuir components
ds.slang

# artificlaly adjust start and stop indices for shotnum 11
ds["id_wstart"].loc[dict(id={"id_signum": 11, "id_swpnum": 0})] = 100 # * 0.1
ds["id_wstop"].loc[dict(id={"id_signum": 11, "id_swpnum": 0})] = 200 # * 0.1

# show dataset
ds

In [ ]:
ds.sel(id=(11, 0))

### Select a Sweep based in its ID

In [ ]:
indexers = ds.slang.sel_indexers(id=(11,0))
print(indexers)
ds.slang.sel(id=(11, 0))
# ds.loc[indexers]

In [ ]:
# How many sweeps does signal number 11 have?
ds.slang.sweep_count(11)

In [ ]:
ds.slang.sel(id=(11, 0)).plot.scatter(x='voltage', y='current')

The sweep window can be overridden

In [ ]:
ds.slang.sel(id=(11, 0), time=slice(None)).plot.scatter(x='voltage', y='current')

In [ ]:
def smooth(a, window_len=10):
    s=np.r_[a[window_len-1:0:-1], a, a[-2:-window_len-1:-1]]
    w = np.ones(window_len, 'd')
    sm = np.convolve(w / w.sum(), s, mode='valid')
    return sm

smooth(ds.slang.sel(id=(11, 0)).current)

In [ ]:
# midx = pd.MultiIndex.from_tuples([(10, 0),
#                                   (11, 0),
#                                   (11, 1)], names=["sid", "tid"])
# midx
# ds.coords["id"] = ("id", midx)
# ds.coords["id_num"] = ("id", np.arange(ds.coords["id"].size))
# ds.coords

def sel_(ds, indexers: Mapping[Hashable, Any] = None, method: str = None, tolerance: Number = None, drop: bool = False, **indexers_kwargs: Any) -> Mapping[Hashable, Any]:
    _shot_index = "shotnum"
    _time_index = "time_index"
    _id_index = "id"
    _id_shot = "s" + _id_index
    _id_trace = "tr" + _id_index
    _id_wstart = _id_index + "_wstart"
    _id_wstop = _id_index + "_wstop"
    
    indexers = either_dict_or_kwargs(indexers, indexers_kwargs, "swept_langmuir.index")

    if _id_index in indexers and _shot_index in indexers:
        print(f"{_id_index} and {_shot_index}")
        
        raise ValueErrors(
            f"I'm not smart enough to resolve indexing using both {_id_index} and "
            f"{_shot_index}, use just one."
        )
    elif _id_index in indexers:
        # generate sel for shot index
        sn = ds[_id_shot].sel({_id_index: indexers[_id_index]}).data
        indexers[_shot_index] = sn
        
        # generate sel for time index
        if _time_index in indexers:
            # let the id time interval be overruled by _time_index
            pass
        else:
            wstart = ds[_id_wstart].sel({_id_index: indexers[_id_index]}).data
            wstop = ds[_id_wstop].sel({_id_index: indexers[_id_index]}).data
            indexers[_time_index] = slice(np.min(wstart), np.max(wstop))
    elif _shot_index in indexers and _time_index in indexers:
        # need to filter for both sid's and trid's
        
        # get shot index values
        sn = indexers[_shot_index]
        
        # get trace index values
        # pick id's where any time value falls between wstart and wstop
#         mask = xr.where((ds[_id_wstart] >= np.min(indexers[_time_index])) 
#                         & (ds[_id_wstop] <= np.max(indexers[_time_index])), True, False).data
        warn(f"I'm not smart enough to select trace id's '{_id_trace}' based on ""
             f"time index '{_time_index}' values.  Only filtering for shot index values", PlasmaPyWarning)
        
    elif _shot_index in indexers:
        # need to filter for sid only
        pass
    elif _time_index in indexers:
        # need to filert for trid only
        pass

#     pos_indexers, new_indexes = remap_label_indexers(
#         ds, indexers=indexers, method=method, tolerance=tolerance
#     )
#     result = ds.isel(indexers=pos_indexers, drop=drop)
#     return result._overwrite_indexes(new_indexes)
#     return ds

#     return ds.sel(indexers=indexers, method=method, tolerance=tolerance, drop=drop)
    return indexers

# ds.sel(id={"sid": 11, "trid": 0})
# indexers = index(ds, id={"sid": 11, "trid": 0})
indexers = sel_(ds, id=([10],))
# sn = ds["sid"].sel(indexers).data
# ds.sel(shotnum=sn)
# ds["id"].sel(id=slice(15))
print(indexers)
ds.sel(indexers).plot.scatter(x="voltage", y="current")

In [ ]:
mask = xr.where((ds.coords["id_wstart"] >= 50) & (ds.coords["id_wstop"] <= 300), True, False).data
ds["trid"][mask].data

In [ ]:
# data = ds.sel(id={"sid": 11, "trid": 0})
data = ds["id"].sel(id=([11, 12], [0]))
# data = ds["id"].isel(id=0)
# data["id"].data.tolist()
# ds

# dir(data["id"].data)


# data["id"].data

data

In [ ]:
ds.coords["pname"] = ("pname", ["vf", "Isat", "Te", "vp"])
ds.coords["pquantity"] = ("pquantity", ["value", "error"])
ds["parameters"] = (("id", "pname", "pquantity"), np.tile(np.nan, (ds["id"].size, ds["pname"].size, ds["pquantity"].size)))
ds

In [ ]:
ds.slang.sel(id=(11,0))


In [ ]:
ds.voltage.dims == ds.current.dims
ds.voltage.shape


In [ ]:
f"voltage {ds.voltage.shape}"

In [ ]:
ds["id"].size

In [ ]:
pd.MultiIndex.from_product([ds["time_index"], [0]], names=["sid", "trid"])

In [ ]:
np.repeat(0, 5)

In [ ]:
ds.coords["id"] = np.arange(ds.current.shape[0])

In [ ]:
# del ds["id"]
# ds.expand_dims("id")
# ds.coords["id"] = ("id", [0])
# ds.assign_coords({
#     "id_istop": ("id", np.array([0])),
# })
ds.coords["id_start"] = ("id", [5])
ds

In [ ]:
names = data.dtype.names
del names['signal']

xa = xr.DataArray(data['signal'], dims=["shotnum", "time"], name="rename")

for name in names:
    if name = 'xyz':
        pass
    else:
        xa.coord[name] = data[name]
    